# `hiRes`

### Chris Hamm
### 2023-08030


A programs to increase the resolution of old digital photos

In [8]:
# Preliminaries

import torch
import cv2
from PIL import Image
import numpy as np
from RealESRGAN import RealESRGAN

# purge cuda memory if needed
torch.cuda.empty_cache()

# If you need to limit memory 
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1028"


In [5]:
# This is from https://github.com/ai-forever/Real-ESRGAN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RealESRGAN(device, scale=4)
model.load_weights('/home/chris/Projects/Real-ESRGAN/weights/RealESRGAN_x4.pth', download=True)

path_to_image = 'input/20160608101900891_0001.jpg'
image = Image.open(path_to_image).convert('RGB')

In [7]:
# enhance and save the  image
sr_image = model.predict(image)

sr_image.save("output/enc_20160608101900891_0001.jpg")

OutOfMemoryError: CUDA out of memory. Tried to allocate 930.00 MiB (GPU 0; 7.92 GiB total capacity; 4.85 GiB already allocated; 725.56 MiB free; 6.63 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Now for movies!

In [7]:
# this one was mostly me, a little pieced to gether by seeing you branched the above repo

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RealESRGAN(device, scale=2) # Note the 2, must match wiht model
model.load_weights('/home/chris/Projects/Real-ESRGAN/weights/RealESRGAN_x2.pth', download=True)
input_video_path = 'input/Monsanto_trimmed.mp4'
output_video_path = 'output/enhanced_Monsanto_trimmed.mp4'

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width * 2, frame_height * 2))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    sr_image = model.predict(image)
    sr_frame = np.array(sr_image)

    # Convert the super-resolved frame back to BGR before writing the video
    sr_frame_bgr = cv2.cvtColor(sr_frame, cv2.COLOR_RGB2BGR)
    
    out.write(sr_frame_bgr)
    
cap.release()
out.release()